### html to markdown

In [1]:
import html2text

def html_to_markdown(html_content):
    # html2text 객체 생성
    markdown_converter = html2text.HTML2Text()

    # 설정: 링크를 단순 텍스트로, 내부 줄바꿈 제거
    markdown_converter.ignore_links = False
    markdown_converter.body_width = 0

    # HTML -> Markdown 변환
    markdown_content = markdown_converter.handle(html_content)

    # 결과 출력
    return markdown_content

In [9]:
import pandas as pd

df = pd.read_excel('documents/QnA_car_sample_30_context_labeling.xlsx')

df = df[df['ground truth page content'].notna()]
df['ground truth page content'] = df['ground truth page content'].apply(html_to_markdown)

df.to_excel('documents/QnA_car_sample_30_context_labeling_markdown.xlsx', index=False)

In [3]:
import json

with open('documents/car_page_upstage_ver.json', 'r') as file:
    data = json.load(file)

data = {k: html_to_markdown(v) for k, v in data.items()}

with open('documents/car_page_upstage_ver_markdown.json', 'w') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [13]:
import json

results_file = 'result_20250131_q_QnA_car_sample_30_context_labeling_markdown.json'
results_path = f'results/{results_file}'

with open(results_path, 'r') as file:
    results = json.load(file)

for method in results[0]['llm response'].keys():
    print(method)


dense_sts_chunking_500_100_rerank_3
dense_sts_chunking_500_100_no_reranker
sparse_bm25_chunking_500_100_rerank_3
sparse_bm25_chunking_500_100_no_reranker
dense_sts_full_page_rerank_3
dense_sts_full_page_no_reranker
sparse_bm25_full_page_rerank_3
sparse_bm25_full_page_no_reranker


In [19]:
# JSON을 DataFrame으로 변환
data = []
for method in results[0]['llm response'].keys():
    for i in range(len(results)):
        data.append({
        "question": results[i]['question'],
        "ground truth answer": results[i]['ground truth answer'],
        "ground truth page contents": results[i]['ground truth answer context'],
        "method": method,
        "rag answer": results[i]['llm response'][method]['answer'],
        "rag answer page contents": results[i]['llm response'][method]['page contents']
    })

df = pd.DataFrame(data)

In [7]:
from bs4 import BeautifulSoup

def html_to_text(html_content):
    soup = BeautifulSoup(html_content, "html.parser")  # Parse HTML
    text = soup.get_text()  # Extract plain text
    return text.replace('\n', ' ')

def clean_text(content):
    return content.replace('\n', ' ')

In [5]:
import json

with open('documents/car_page_upstage_ver.json', 'r') as file:
    data = json.load(file)

data = {k: html_to_text(v) for k, v in data.items()}

with open('documents/car_page_upstage_ver_text.json', 'w') as file:
    json.dump(data, file, ensure_ascii=False)

In [37]:
import pandas as pd

df = pd.read_excel('documents/QnA_car_sample_30_context_labeling.xlsx')
df = df.dropna(subset=['ground truth page content'])
df = df.assign(**{
    'ground truth page content': df['ground truth page content'].fillna('').apply(html_to_text)
})
df.to_excel('documents/QnA_car_sample_30_context_labeling_text.xlsx', index=False)

### HyDE

In [19]:
from modules.config_loader import load_config
import os

config = load_config('config/config.yaml')

os.environ['LANGCHAIN_TRACING_V2'] = config['langchain']['tracing_v2']
os.environ['LANGCHAIN_ENDPOINT'] = config['langchain']['endpoint']
os.environ['LANGCHAIN_API_KEY'] = config['langchain']['api_key']

os.environ['OPENAI_API_KEY'] = config['openai']['api_key']

In [52]:
from langchain.prompts import ChatPromptTemplate

prompt = """해당 질문에 답하기 위한 보험 약관 단락을 작성해주세요.
질문: {question}
단락:"""
prompt_hyde = ChatPromptTemplate.from_template(prompt)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=config['openai']['gpt_model'], temperature=0)

hyde_chain = prompt_hyde | llm | StrOutputParser()

question = "태풍으로 인한 차량피해를 입은 경우 보험처리가 되나요?"
generated_docs_for_retrieval = hyde_chain.invoke({"question": question})
print(generated_docs_for_retrieval)

"고객님께서 가입하신 자동차 보험 약관에 따라, 태풍과 같은 자연재해로 인한 차량 피해는 보상 대상에 포함될 수 있습니다. 자연재해로 인한 손해는 일반적으로 '자기차량손해' 담보에 의해 보상되며, 이는 고객님께서 해당 담보를 선택하여 가입하신 경우에 한합니다. 보상 범위와 한도는 가입하신 보험 상품의 세부 조건에 따라 다를 수 있으므로, 구체적인 보상 가능 여부와 절차에 대해서는 보험 증권 및 약관을 참조하시거나, 당사 고객센터로 문의해 주시기 바랍니다."

In [53]:
from modules.dense_retrieval import CustomEmbeddings
from modules.document_processor import DocumentProcessor
from modules.qa_chain import QAChain
import torch

doc_processor = DocumentProcessor()
docs, metadata = doc_processor.load_and_process_documents(
    file_path=config['dataset']['document'],
    mode=config['dataset']['document_type'],
    do_chunking=config['preprocessing']['chunking'],
    chunk_size=config['preprocessing']['chunk_size'],
    chunk_overlap=config['preprocessing']['chunk_overlap']
)

faiss_index_path = config['dataset']['document'].replace(
    '.json',
    f"_{config['dense_model']['model_type']}_" +  # sts 또는 api 구분
    ('chunk.bin' if config['preprocessing']['chunking'] else 'full.bin')
)
embedding_service = CustomEmbeddings(
    model_type=config['dense_model']['model_type'],
    model_path=config['dense_model']['model_cache'],
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    embedding_config=config,
        faiss_index_path=faiss_index_path
    )
retriever = embedding_service.create_or_load_vectorstore(docs)

In [56]:
retrieval_chain = hyde_chain | retriever.as_retriever()

retrieved_docs = retrieval_chain.invoke({"question": question})

for doc in retrieved_docs:
    print(doc.page_content)
    print(doc.metadata)

중 생긴 사고로 인한 손해제5조(보험금의 청구시 제출서류)피보험자가 보험금을 청구하고자 할 때에는 다음의 서류 등을 구비하여 보험금을 청구하여야 합니다.1. 보험금 청구서2. 손해액을 증명하는 서류(진단서 등)3. 사고가 발생한 때와 장소 및 사고사실이 신고된 관할 경찰서4. 배상의무자의 주소, 성명 또는 명칭, 차량번호5. 배상의무자의 「대인배상Ⅱ」 또는 공제계약의 유무 및 내용6. 피보험자가 입은 손해를 보상할 「대인배상Ⅱ」 또는 공제계약, 배상의무자또는 제3자로부터 이미 지급받은 손해배상금이 있을때 그 금액7. 그 밖에 보험회사가 꼭 필요하여 요청하는 서류 등제6조(준용규정)이 특별약관에서 정하지 아니한 사항은 보통약관에 따릅니다.104 특별약관간병비지원 특별약관제1조(적용대상)이 특별약관은 보통약관의 「자기신체사고」 또는 「자동차상해 특별약관」을 가입하고 「간병비지원 특별약관」에 가입한 경우에 적용됩니다.제2조(보상내용)보험회사는 피보험자가 피보험자동차를 소유, 사용,
{'page': '54'}
「자기신체사고」 또는 「자동차상해 특별약관」, 「무보험자동차에 의한 상해」, 「자기차량손해」 또는 「차량단독사고 손해보상 특별약관」의 손해에 대하여는 보험금을 지급합니다. 또한, 다음 중 어느 하나에 해당하는 손해에 대하여는 보험금을 지급합니다.1. 피보험자동차를 도난당하였을 경우 그 도난당하였을 때로부터 발견될 때까지의 사이에 발생된 피보험자동차의 사고로 인한 보통약관 「대인배상Ⅱ」,「대물배상」 또는 「대물배상 가입금액 확장특별약관」, 「자기신체사고」 또는「자동차상해 특별약관」, 「무보험자동차에 의한 상해」, 「자기차량손해」 또는「차량단독사고 손해보상 특별약관」의 손해2. 관련법규에 의해 사업자등록을 한 자동차 취급업자가 업무상 위탁받은 피보험자동차를 사용하거나 관리하던 중 발생된 피보험자동차의 사고로 인한보통약관 「대인배상Ⅱ」, 「대물배상」에서 보상하는 손해. 다만, 자동차 취급업자가 가입한 보험계약에서 보험금이 지급될 수 있는 경우에는 그 보험금을 초과하는

In [55]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retrieved_docs,"question":question})

'제공된 문서에서는 태풍과 같은 자연재해로 인한 차량 피해에 대한 보험 처리 여부에 대한 정보가 명시되어 있지 않습니다. 일반적으로 자동차 보험 약관은 자연재해로 인한 손해를 보상하는지 여부를 명확히 규정하고 있으며, 이는 보험 상품과 약관에 따라 다를 수 있습니다. 따라서, 태풍으로 인한 차량 피해가 보험 처리 대상인지 확인하려면 해당 보험의 약관을 직접 확인하거나 보험사에 문의하는 것이 좋습니다.'